In [ ]:
import redframes as rf
import pandas as pd
import numpy as np

In [ ]:
df = rf.load("nfl.csv")

(
    df.filter(lambda row: (
        row["Team"].isin(["BUF", "KCC"]) & 
        (row["Points"] >= 100)
    ))
    .sort(["Points"], reverse=True)
    .cumsum
)

In [ ]:
df = pd.read_csv("nfl.csv")

In [ ]:
df.assign(cum_average=df.groupby('Team', sort=False)['Points'].transform(lambda x: x.expanding().mean()))

In [ ]:
rf.export(df, "example.csv")
df.save("")

In [ ]:
# group by
# take
# mutate
# cum sum
# aggregate

from typing import Literal

# df.sort_values("Points", ascending=False).groupby(["Position"]).head(4).cummean()

df = pd.read_csv("nfl.csv")

def accumulate(df: pd.DataFrame, column: str, into: str, method: Literal["min", "max", "sum"] = "sum") -> pd.DataFrame:
    df = df.copy()
    if method == "min":
        df[into] = df[column].cummin()
    elif method == "max":
        df[into] = df[column].cummax()
    elif method == "sum":
        df[into] = df[column].cumsum()
    else:
        raise ValueError("method is invalid, must be one of {'min', 'max', 'sum'}")
    return df

accumulate(df, "Points", into="cumsum_points", method="sum")

In [1]:

import redframes as rf
import pandas as pd

df = pd.read_csv("nfl.csv")

apropos = {
    # "count": ("Points", rf.compute.count),
    "sum": ("Points", rf.compute.sum),
    "mean": ("Points", rf.compute.mean),
    "median": ("Points", rf.compute.median), 
    "min": ("Points", rf.compute.min), 
    "max": ("Points", rf.compute.max),
    "std": ("Points", rf.compute.std)
}
df.groupby(["Team", "Position"]).agg(**apropos).reset_index()

,Team,Position,sum,mean,median,min,max,std
0,ARI,D,95.7,95.700000,95.70,95.7,95.7,0.000000
1,ARI,K,127.2,127.200000,127.20,127.2,127.2,0.000000
2,ARI,QB,382.0,191.000000,191.00,28.9,353.1,162.100000
3,ARI,RB,406.1,101.525000,72.65,9.0,251.8,97.238094
4,ARI,TE,128.6,42.866667,33.20,7.4,88.0,33.607274
...,...,...,...,...,...,...,...,...
186,WAS,K,114.9,114.900000,114.90,114.9,114.9,0.000000
187,WAS,QB,231.8,115.900000,115.90,64.0,167.8,51.900000
188,WAS,RB,360.4,90.100000,70.40,11.0,208.6,76.509019
189,WAS,TE,118.6,59.300000,59.30,30.7,87.9,28.600000
